In [ ]:
!pip install easyocr
!pip install pdf2image
!sudo apt-get install poppler-utils

from pdf2image import convert_from_path
import numpy as np
import easyocr, cv2

!git clone https://github.com/Chaitanya-Thombare/OCR-Text_Extraction

In [12]:
def process_img(raw_frame, scale):
    #gray_frame = cv2.cvtColor(raw_frame, cv2.COLOR_BGR2GRAY)
    scaled_gray_frame = cv2.resize(raw_frame, (raw_frame.shape[1]//scale, raw_frame.shape[0]//scale))
    return scaled_gray_frame

In [23]:
def detect_and_extract_text(blank_frame):
    text_data = []
    reader = easyocr.Reader(['en'])

    res = reader.readtext(blank_frame)

    for (bbox, text, prob) in res: 
        # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        br = (int(br[0]), int(br[1]))

        text_data.append([[(tl), (br)], text])
        text_list.append((text))
        detections_frame = cv2.rectangle(blank_frame, tl, br, (0, 255, 0), 2)

    return detections_frame, text_data

In [5]:
def structure_data(text_data, width):
    grp_1, grp_2, grp_3 = [], [], []

    for box in text_data:
        box_mid = (box[0][0][0] +  box[0][1][0])/2
        if 0.00 * width <= box_mid < 0.33 * width:
            grp_1.append(box)
        elif 0.33 * width <= box_mid < 0.66 * width:
            grp_2.append(box)
        elif 0.66 * width <= box_mid < 1.00 * width:
            grp_3.append(box)
    
    grp_1.sort(key=lambda x: x[0][0][1])
    grp_2.sort(key=lambda x: x[0][0][1])
    grp_3.sort(key=lambda x: x[0][0][1])

    final_text = []
    final_text.extend([box[1] for box in grp_1])
    final_text.extend([box[1] for box in grp_2])
    final_text.extend([box[1] for box in grp_2])

    return final_text

In [14]:
def read_text(undetected_frame):
    detections_frame, text_data = detect_and_extract_text(blank_frame=frame)
    print("Texts detected")
    raw_text = structure_data(text_data=text_data, width=detections_frame.shape[1])
    all_text = '//'.join(raw_text)

    return raw_text, all_text, detections_frame

In [ ]:
assert False

AssertionError: ignored

In [ ]:
pdf_path = '/content/drive/MyDrive/Box/Immunity guide(1).pdf'
#images = convert_from_path("OCR9.jpg")
%cd output
for i in range(len(images)):
    frame = process_img(raw_frame=np.array(images[i]), scale=2)
    raw_text, text, marked_frame = read_text(frame)

    img_name = "PDF_image_" + str(i) + ".jpg"
    cv2.imwrite(img_name, marked_frame)

    file_name = "PDF_text_" + str(i) + ".txt"
    text_file = open(file_name,"a")
    text_file = text_file.write(text)